# Distribution Invention: Initial Exploration
Testing our setup and exploring initial ideas

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras import layers, ops

print(f"Keras version: {keras.__version__}")
print(f"Backend: {keras.backend.backend()}")

In [ ]:
# Simple test: Can we create a layer that modifies its behavior?
class SimpleDistributionModifier(layers.Layer):
    def __init__(self, units=32, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        
    def build(self, input_shape):
        self.base_kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True,
            name='base_kernel'
        )
        self.modifier = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='zeros',
            trainable=True,
            name='modifier'
        )
        
    def call(self, inputs, modification_strength=0.0):
        # Apply base transformation
        base_output = ops.matmul(inputs, self.base_kernel)
        
        # Apply modification based on strength
        modified_kernel = self.base_kernel + modification_strength * self.modifier
        modified_output = ops.matmul(inputs, modified_kernel)
        
        return modified_output

# Test the layer
layer = SimpleDistributionModifier(16)
test_input = ops.ones((2, 8))
output_base = layer(test_input, modification_strength=0.0)
output_modified = layer(test_input, modification_strength=1.0)

print(f"Base output shape: {output_base.shape}")
print(f"Output difference: {ops.mean(ops.abs(output_modified - output_base))}")